In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import random
import time
import tensorflow as tf

In [2]:
n_train = 2320
n_valid = 1048

In [3]:
IMG_SIZE = 160

def load(dir, n):
    x_data, y_data = [], []
    for i in range(n):
        image = cv2.imread(dir + "/" + str(i + 1) + ".png")
        r = open(dir + "/" + str(i + 1) + ".txt", "r")
        label = int(r.readline())
        image = tf.cast(image, tf.float32)
        image = (image/127.5) - 1
        x_data.append(image)
        y_data.append(label)
    return x_data, y_data

X_train, Y_train = load("/home/dima/PycharmProjects/Practics/Dataset/train", n_train)
X_valid, Y_valid = load("/home/dima/PycharmProjects/Practics/Dataset/valid", n_valid)
Y_train = tf.cast(Y_train, tf.int32)
Y_valid = tf.cast(Y_valid, tf.int32)
Y_train -= 1
Y_valid -= 1
print(Y_train)
X_train_norm = tf.norm(X_train)
X_valid_norm = tf.norm(X_valid)

#X_train = X_train / X_train_norm
#X_valid = X_valid / X_valid_norm


tf.Tensor([0 0 0 ... 3 3 3], shape=(2320,), dtype=int32)


In [4]:
train = tf.data.Dataset.from_tensor_slices( (X_train, Y_train) )
valid = tf.data.Dataset.from_tensor_slices( (X_valid, Y_valid) )

In [5]:
print(train)
print(valid)

<TensorSliceDataset shapes: ((160, 160, 3), ()), types: (tf.float32, tf.int32)>
<TensorSliceDataset shapes: ((160, 160, 3), ()), types: (tf.float32, tf.int32)>


In [6]:
BATCH_SIZE = 29
SHUFFLE_BUFFER_SIZE = 1000

In [7]:
train_batches = train.batch(BATCH_SIZE)
validation_batches = valid.batch(BATCH_SIZE)
print(train_batches, validation_batches)

<BatchDataset shapes: ((None, 160, 160, 3), (None,)), types: (tf.float32, tf.int32)> <BatchDataset shapes: ((None, 160, 160, 3), (None,)), types: (tf.float32, tf.int32)>


In [8]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

TensorShape([29, 160, 160, 3])

In [9]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)


base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [10]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)


(29, 5, 5, 1280)


In [11]:
base_model.trainable = False
base_model.summary()


Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [12]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)


(29, 1280)


In [13]:
import tensorflow.keras as keras

prediction_layer = keras.layers.Dense(4)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)


(29, 4)


In [14]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 5124      
Total params: 2,263,108
Trainable params: 5,124
Non-trainable params: 2,257,984
_________________________________________________________________


In [15]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss='mean_squared_error',
              metrics=['accuracy'])


In [16]:
print(len(model.trainable_variables))
for image_batch, label_batch in validation_batches:
    print(label_batch)

2
tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3], shape=(29,), dtype=int32)
tf.Tensor([3 3 3 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2], shape=(29,), dtype=int32)
tf.Tensor([2 2 2 2 2 2 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2], shape=(29,), dtype=int32)
tf.Tensor([2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 1 1 1 1], shape=(29,), dtype=int32)
tf.Tensor([1 1 1 1 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2], shape=(29,), dtype=int32)
tf.Tensor([2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3], shape=(29,), dtype=int32)
tf.Tensor([3 3 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 2 2 2], shape=(29,), dtype=int32)
tf.Tensor([2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1], shape=(29,), dtype=int32)
tf.Tensor([0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3 3], shape=(29,), dtype=int32)
tf.Tensor([3 3 3 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3], shape=(29,), dtype=int32)
tf.Tensor([3 3 3 3 3 3 3 3 3

In [17]:
num_train = 290

initial_epochs = 10
steps_per_epoch = round(num_train)//BATCH_SIZE
validation_steps = 50
model.evaluate(validation_batches, steps = 37)
predictions = model.predict(validation_batches, steps = validation_steps, verbose = 1)
print(predictions)

37/50 [=====================>........] - ETA: 9s WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 50 batches). You may need to use the repeat() function when building your dataset.
[[ 2.248071   -1.1797987   0.29056114 -0.37479565]
 [ 1.23281    -0.88217866  1.3003064  -1.2871965 ]
 [ 1.4793293  -1.5714374   0.36898708 -0.20814992]
 ...
 [ 0.81751037 -2.3091848   0.02200836  1.2666864 ]
 [ 1.5821323  -2.0696511   1.1872894   2.0563061 ]
 [ 1.4015552  -2.4440448   0.7009318   1.6176902 ]]


In [18]:

history = model.fit(train_batches,
                    epochs=initial_epochs, validation_data=validation_batches)


Epoch 1/10
80/80 [==============================] - 129s 2s/step - loss: 2.0733 - accuracy: 0.3703 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
80/80 [==============================] - 127s 2s/step - loss: 1.6896 - accuracy: 0.2470 - val_loss: 2.1566 - val_accuracy: 0.1002
Epoch 3/10
80/80 [==============================] - 123s 2s/step - loss: 1.5952 - accuracy: 0.2474 - val_loss: 2.0357 - val_accuracy: 0.1040
Epoch 4/10
80/80 [==============================] - 120s 1s/step - loss: 1.5186 - accuracy: 0.2470 - val_loss: 1.9371 - val_accuracy: 0.1040
Epoch 5/10
80/80 [==============================] - 121s 2s/step - loss: 1.4554 - accuracy: 0.2461 - val_loss: 1.8562 - val_accuracy: 0.1040
Epoch 6/10
80/80 [==============================] - 120s 2s/step - loss: 1.4025 - accuracy: 0.2448 - val_loss: 1.7895 - val_accuracy: 0.1031
Epoch 7/10
80/80 [==============================] - 120s 1s/step - loss: 1.3575 - accuracy: 0.2453 - val_loss: 1.7343 - val_accuracy: 0.1031
Epoch

In [19]:
predictions = model.predict(validation_batches, steps = validation_steps, verbose = 1)
print(predictions)

37/50 [=====================>........] - ETA: 8sWARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 50 batches). You may need to use the repeat() function when building your dataset.
[[ 2.059375    1.298714    1.3425223   0.7462917 ]
 [ 1.0811696   1.0253167   1.6540525  -0.35365808]
 [ 1.4903767   0.6523695   1.2578924   0.87826884]
 ...
 [ 0.85462487  0.12666899  0.92593646  2.0438092 ]
 [ 0.9994313   0.02714214  1.6151617   2.325631  ]
 [ 1.0510422   0.0207153   1.468354    2.2782018 ]]


In [20]:
for p in predictions:
    print(p)

[2.059375  1.298714  1.3425223 0.7462917]
[ 1.0811696   1.0253167   1.6540525  -0.35365808]
[1.4903767  0.6523695  1.2578924  0.87826884]
[1.029931   1.1627531  1.7803731  0.01708951]
[1.5662363  1.7508776  1.9939808  0.17586803]
[ 0.94156885  0.39366862  2.1039505  -0.32435164]
[1.0040065  1.7146988  1.5904404  0.90620404]
[ 0.78253907  0.7919411   1.3609377  -0.41114005]
[3.004356  1.8496122 3.3112159 1.8945034]
[1.1294665 1.3766508 3.4477687 0.352559 ]
[1.4248013  1.6253512  2.3761148  0.81223047]
[0.88508856 1.0849545  1.7454116  1.7395797 ]
[1.5434846  0.4859758  1.8568504  0.76500356]
[1.2336352  1.2566988  2.8987205  0.42753035]
[1.8753898 2.3792603 1.7825272 1.2091396]
[1.188289  1.772174  1.0839154 1.6012971]
[2.157189   0.51771474 2.067465   2.4441898 ]
[1.5427177 0.9739114 2.2493677 1.8277317]
[ 2.2053242  -0.11817043  3.05214     2.3322027 ]
[1.8048633 0.090186  2.2051191 1.6397357]
[1.3770204 0.5406438 2.4372938 1.4012969]
[0.79913145 0.61568075 2.695666   0.90161467]
[1.8

[2.0022023 1.5421853 2.8822274 1.3775368]
[2.3260264 1.1508744 2.2910879 1.9534765]
[2.017337   0.25694424 3.1423082  1.0008396 ]
[1.7409029 1.0115895 2.3324041 1.1781029]
[1.8351407  0.8826922  2.4456086  0.43885848]
[2.8043935 2.01734   2.9591687 2.5566497]
[1.1830812 1.3658586 2.1081247 1.6072824]
[1.891722  1.680412  2.9107537 1.644084 ]
[1.5896004  0.9115864  2.5479295  0.56378084]
[1.7952867 1.6633632 3.6403887 2.23182  ]
[1.8157754 1.3598151 2.3635547 1.3800817]
[2.2861521 2.4346423 4.0225167 1.6956708]
[1.5122222 1.4597695 2.821116  0.7184849]
[1.7798914  1.6231661  2.3811743  0.93050086]
[ 1.2879338   1.4279656   3.368876   -0.07041887]
[1.6697459 1.2272354 2.867879  1.1850642]
[0.6425483 1.8037543 2.9108639 1.6409843]
[2.7209811  0.10199178 3.5292356  0.86040866]
[1.5591843  0.80403066 3.295941   1.0217189 ]
[1.6078899  1.1819942  2.8203318  0.97989285]
[1.313175   1.9613353  2.7858531  0.99810356]
[1.6992471 2.1328988 1.3625841 1.8483967]
[1.3674059 1.7345842 1.7258897 1.042

[1.4376117 1.0125761 2.7436156 1.1772952]
[1.2966237 1.874616  2.564779  1.8507489]
[1.8856206  1.365418   1.7780906  0.17340463]
[0.49553722 0.9696698  2.393057   0.91542983]
[0.7574306 1.2505172 1.7788774 1.7317656]
[1.1055413 1.2158693 1.9267665 1.4556432]
[1.0698273  0.75903374 2.6170154  1.1399361 ]
[0.66124445 0.8844948  2.457634   0.49246302]
[0.7393786  0.9542047  1.76768    0.48414272]
[0.8245316 1.2809945 2.2743258 1.3354104]
[1.180167   1.5398145  2.0422325  0.83418226]
[0.3725211 1.4031621 2.1564825 2.0601358]
[0.9717901  0.96398675 2.454661   1.4281579 ]
[1.5257277  0.69839394 2.158218   1.6583321 ]
[1.450755  1.702749  3.1466281 1.862798 ]
[1.2245263 1.3929824 1.274601  0.9908502]
[1.9751927 2.3840535 2.1358287 1.5635884]
[1.2025099  0.61403143 1.9699557  1.6890414 ]
[1.5106192 1.7076613 2.886803  0.0698471]
[1.2582502  1.3114078  1.6525967  0.65170085]
[1.9195143  0.69391084 2.2502937  0.23131935]
[0.8275137  1.0672723  1.5482123  0.25946984]
[2.1145906  1.858084   2.885

[1.5428008 0.8599844 2.405087  1.7953769]
[0.70339143 1.33057    2.557709   1.9530364 ]
[1.7146616 1.1455854 1.0786742 1.2414544]
[1.0713232  0.97930884 1.3278269  1.2405177 ]
[1.1576264 0.4296716 1.3912275 2.4792225]
[0.85462487 0.12666899 0.92593646 2.0438092 ]
[0.9994313  0.02714214 1.6151617  2.325631  ]
[1.0510422 0.0207153 1.468354  2.2782018]


In [21]:
base_model.trainable = True

print("Number of layers in the base model: ", len(base_model.layers))

fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False


Number of layers in the base model:  155


In [22]:
model.compile(loss='mean_squared_error',
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 5124      
Total params: 2,263,108
Trainable params: 1,867,716
Non-trainable params: 395,392
_________________________________________________________________


In [23]:
len(model.trainable_variables)

58

In [24]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch =  history.epoch[-1],
                         validation_data=validation_batches)


Epoch 10/20
80/80 [==============================] - 166s 2s/step - loss: 1.2114 - accuracy: 0.2427 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 11/20
80/80 [==============================] - 164s 2s/step - loss: 0.9560 - accuracy: 0.2409 - val_loss: 1.4513 - val_accuracy: 0.0916
Epoch 12/20
80/80 [==============================] - 157s 2s/step - loss: 0.7999 - accuracy: 0.2341 - val_loss: 1.4283 - val_accuracy: 0.0926
Epoch 13/20
80/80 [==============================] - 156s 2s/step - loss: 0.6749 - accuracy: 0.2353 - val_loss: 1.4252 - val_accuracy: 0.0935
Epoch 14/20
80/80 [==============================] - 158s 2s/step - loss: 0.5713 - accuracy: 0.2414 - val_loss: 1.4293 - val_accuracy: 0.0916
Epoch 15/20
80/80 [==============================] - 157s 2s/step - loss: 0.4837 - accuracy: 0.2392 - val_loss: 1.4319 - val_accuracy: 0.0887
Epoch 16/20
80/80 [==============================] - 156s 2s/step - loss: 0.4091 - accuracy: 0.2401 - val_loss: 1.4345 - val_accuracy: 0.089